In [ ]:
# ! pip uninstall -y nemo_guardrails

# Trace Recording Example

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.parent.resolve()))

In [2]:
from concurrent.futures import as_completed
from time import sleep

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from tqdm.auto import tqdm

from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval.feedback.provider.dummy import DummyProvider
from trulens_eval.schema.feedback import FeedbackMode
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

tp = TP()

d = DummyProvider(
    loading_prob=0.0,
    freeze_prob=0.0, # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.0,
    overloaded_prob=0.0,
    rpm=1000,
    alloc = 0, # how much fake data to allocate during requests
    delay = 10.0
)

tru = Tru()

tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
Force stopping dashboard ...
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://172.17.97.160:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [3]:
# Create custom app:
ca = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [4]:
with ta as recorder:
    res = ca.respond_to_query(f"hello")

rec = recorder.get()
assert rec is not None

tracer SpanRecord
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanMethodCall
tracer SpanCost
tracer SpanMethodCall
tracer SpanMethodCall
finished recording, have:
9a/4a SpanRecord
9a/ee SpanMethodCall
9a/2a SpanMethodCall
9a/83 SpanMethodCall
9a/78 SpanMethodCall
9a/6e SpanMethodCall
9a/f2 SpanMethodCall
9a/19 SpanMethodCall
9a/5d SpanMethodCall
9a/fe SpanMethodCall
9a/ed SpanMethodCall
9a/e4 SpanMethodCall
9a/f2 SpanMethodCall
9a/82 SpanCost
9a/a8 SpanMethodCall
9a/1e SpanMethodCall


In [5]:
for context, span in recorder.tracer[1].spans.items():
    print(span.parent, "->", context, type(span).__name__, end="")
    if hasattr(span, "call"):
        print("\t", span.call.top().path, span.call.top().method.name)
        print("\t", span.error)
    elif hasattr(span, "cost"):
        print("\t", span.cost)
    else:
        print()

None -> 9a/4a SpanRecord
9a/4a -> 9a/ee SpanMethodCall	 app respond_to_query
	 None
9a/ee -> 9a/2a SpanMethodCall	 app.agents[0] invoke
	 None
9a/ee -> 9a/83 SpanMethodCall	 app.agents[1] invoke
	 None
9a/ee -> 9a/78 SpanMethodCall	 app get_context
	 None
9a/78 -> 9a/6e SpanMethodCall	 app.retriever retrieve_chunks
	 None
9a/78 -> 9a/f2 SpanMethodCall	 app.reranker rerank
	 None
9a/78 -> 9a/19 SpanMethodCall	 app process_chunk_by_random_tool
	 None
9a/19 -> 9a/5d SpanMethodCall	 app.tools[1] invoke
	 None
9a/78 -> 9a/fe SpanMethodCall	 app process_chunk_by_random_tool
	 None
9a/fe -> 9a/ed SpanMethodCall	 app.tools[0] invoke
	 None
9a/ee -> 9a/e4 SpanMethodCall	 app.memory remember
	 None
9a/ee -> 9a/f2 SpanMethodCall	 app.llm generate
	 None
9a/f2 -> 9a/82 SpanCost	 n_requests=1 n_successful_requests=1 n_classes=0 n_tokens=14 n_stream_chunks=0 n_prompt_tokens=5 n_completion_tokens=9 cost=0.0159
9a/ee -> 9a/a8 SpanMethodCall	 app.template fill
	 None
9a/ee -> 9a/1e SpanMethodCall	 app.

In [ ]:
from typing import Self, Type, Iterator, Generator, AsyncGenerator, Awaitable, AsyncIterator
import asyncio
import time

from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import (
    BatchSpanProcessor,
    ConsoleSpanExporter,
)

trace.set_tracer_provider(TracerProvider())
trace.get_tracer_provider().add_span_processor(
    BatchSpanProcessor(ConsoleSpanExporter())
)

tracer = trace.get_tracer(__name__)

In [ ]:
import functools

def instrument(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        with tracer.start_as_current_span(func.__name__) as span:
            span.set_attribute("args", str(args))
            span.set_attribute("kwargs", str(kwargs))

            ret = func(*args, **kwargs)

            span.set_attribute("ret", str(ret))

            return ret

    return wrapper

from opentelemetry import trace
tracer = trace.get_tracer(__name__)
print(tracer)

@tracer.start_as_current_span("innerfunction")
def innerfunction(a: int) -> int:

    current_span = trace.get_current_span()
    current_span.set_attribute("input_a", a)

    ret = a + 1

    current_span.set_attribute("ret", ret)

    return ret


@tracer.start_as_current_span("outerfunction")
def outerfunction(a: int) -> int:

    current_span = trace.get_current_span()
    current_span.set_attribute("input_a", a)

    ret = innerfunction(a) * 2

    current_span.set_attribute("ret", ret)

    return ret

# Cannot use decorator while establishing parent-child relationship to the inner
# spans.
def somegenfunction(a: int) -> Iterator[int]:
    with tracer.start_as_current_span("somegenfunction") as current_span:
        current_span.set_attribute("input_a", a)

        for i in range(a):
            with tracer.start_as_current_span("somegenfunction_iterations") as iter_span:

                iter_span = trace.get_current_span()
                iter_span.set_attribute("iteration", i)

                time.sleep(1)
                
                yield i
        
@tracer.start_as_current_span("someasyncfunction")
async def someasyncfunction(a: int) -> int:
    current_span = trace.get_current_span()
    current_span.set_attribute("input_a", a)
    await asyncio.sleep(1)

    ret = a + 1

    current_span.set_attribute("ret", ret)

    return ret

# Same decorator problem.
async def someasyncgenfunction(a: int) -> AsyncIterator[int]:

    with tracer.start_as_current_span("someasyncgenfunction") as current_span:
        current_span.set_attribute("input_a", a)

        for i in range(a):
            with tracer.start_as_current_span("someasyncgenfunction_iterations") as iter_span:
                iter_span = trace.get_current_span()
                iter_span.set_attribute("iteration", i)

                await asyncio.sleep(1)

                yield i

class SomeClass(object):
    # @tracer.start_as_current_span("somemethod")
    @instrument
    def somemethod(self, a: int) -> int:
        # current_span = trace.get_current_span()
        # current_span.set_attribute("input_a", a)

        return a + 2
    
    #@tracer.start_as_current_span("somestaticmethod")
    @instrument
    @staticmethod
    def somestaticmethod(a: int) -> int:

        #current_span = trace.get_current_span()
        #current_span.set_attribute("input_a", a)

        return a + 3
    
    # @tracer.start_as_current_span("someclassmethod")
    @instrument
    @classmethod
    def someclassmethod(cls: Type[Self], a: int) -> int:

        current_span = trace.get_current_span()
        current_span.set_attribute("input_a", a)

        return a + 4

#with tracer.start_as_current_span("root") as span_root:
#    span_root.set_attribute("root_someattr", 42)
#    print("root hello")
#    with tracer.start_as_current_span("level1") as span_level1:
#        span_level1.set_attribute("level1_someattr", 100)
#        print("level1 hello")

In [ ]:
dir(tracer)

In [ ]:
with tracer.start_as_current_span("root") as span:
    innerfunction(1)
    innerfunction(1)
#outerfunction(4)

# await someasyncfunction(4)

# for a in somegenfunction(3):
#    print(a)

# await someasyncfunction(2)
# async for a in someasyncgenfunction(4):
#     print(a)

In [ ]:
dir(tracer)

In [ ]:
from trulens_eval.trace import Tracer, TracerProvider, tracer_provider
tp = tracer_provider

with tp.trace() as tracer:
    with tracer.method() as span1:
        print(type(span1), span1)

    with tp.trace() as inner_tracer:

        with tracer.method() as span2:
            print(type(span2), span2)
            raise Exception("test outer tracer error")

        with inner_tracer.method() as span3:
            print(type(span3), span3)
            raise Exception("test inner tracer error")

In [ ]:
from trulens_eval.trace import Tracer, TracerProvider, tracer_provider, get_tracer

tracer = get_tracer()
inner_tracer = get_tracer()

with tracer.method() as span1:
    print(type(span1), span1)

    with tracer.method() as span2:
        print(type(span2), span2)

    with inner_tracer.method() as span3:
        print(type(span3), span3)